In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("spotify_millsongdata.csv")

In [22]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [23]:
df.shape

(57650, 4)

In [24]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [25]:
df.head(10)


,artist,song,text
0,Britney Spears,Love To Hate You,Love to hate you \r\n \r\nI'm doing fine \r...
1,Ian Hunter,Ships,"(ian hunter) \r\n \r\nWe walked to the sea, ..."
2,"Harry Connick, Jr.",Honestly Now,Honestly now... \r\nEither you stand for some...
3,Yngwie Malmsteen,Fury,I now will tell you all a story \r\nThat no o...
4,Cher,Believe (Club 69 Phunk Dub),No matter how hard I try \r\nYou keep pushing...
5,Madonna,Laugh To Keep From Crying,"I can't lock my door \r\nOh, I don't have no ..."
6,Carpenters,Ordinary Fool,Only a fool like fools before me \r\nI always...
7,Linkin Park,Wisdom Justice And Love,I come to this magnificent house of worship to...
8,Hank Snow,Hello Love,Well look who's comin' through the door \r\nI...
9,Barbra Streisand,"Hello, Dolly! Finale","[Horace:] \r\nHello, Dolly! \r\nWell, hello,..."


In [26]:
df['text'][0]


"Love to hate you  \r\n  \r\nI'm doing fine  \r\nI'm doing fine  \r\n  \r\nFor seven months and fifteen days  \r\nI've been telling you lies in different ways  \r\nI know things will change  \r\nBut I don't know when  \r\nYou keep talking about us  \r\nLike we still were friends  \r\n  \r\nI just hate the way you talk to me  \r\nI just love to say what's bothering me  \r\n  \r\nI love to hate you  \r\nI really love to hate you  \r\nAnd nothing you do  \r\nWill ever change my mind  \r\nI'm doing fine  \r\nWithout you  \r\n  \r\nIt wasn't meant to be like this  \r\nAll you gave her was one little innocent kiss  \r\nI didn't believe  \r\nShe would turn to be  \r\nThe love of your life  \r\nJust like me  \r\n  \r\nI just hate the way you talk to me  \r\nI just love to say what you mean to me  \r\n  \r\nI love to hate you  \r\nI really love to hate you  \r\nAnd nothing you do  \r\nWill ever change my mind  \r\nI'm doing fine  \r\nWithout you  \r\n  \r\nI'm doing fine  \r\n  \r\nI love to ha

In [27]:
df.shape

(5000, 3)

In [28]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [29]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [30]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [60]:
similarity[0]

array([1.        , 0.03898525, 0.03504442, ..., 0.19384366, 0.02661255,
       0.02913636])

In [64]:
df[df['song'] == 'Love To Hate You']

,artist,song,text
0,Britney Spears,Love To Hate You,love to hate you i 'm do fine i 'm do fine for...


In [68]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs,distances[1:21]

In [70]:
recommendation('Love To Hate You')

(['Outside Of That I Love You',
  'Love, Hate, Love',
  "Love's In Need Of Love Today",
  'I Hate You',
  'Love Is War',
  'Not In That Way',
  'Who Do You Love',
  'Doing Fine',
  'Stay With Me Forever',
  'The Pros And Cons Of Breathing',
  'Now Mary',
  'All You Need Is Love',
  'Make Believe',
  'All My Life',
  "How I Learned To Hate Rock 'n' Roll",
  'Just One Way To Say I Love You',
  'Love Me Again',
  'Looking For Love',
  'Soul Creation By Cinder',
  'Change'],
 [(670, 0.5806907534678949),
  (3359, 0.5032470305681955),
  (3698, 0.3823703552005879),
  (2370, 0.3680895327910472),
  (292, 0.330415491730018),
  (257, 0.3276808380284808),
  (3902, 0.3182029436574636),
  (2921, 0.3160327623495501),
  (398, 0.3100810076709125),
  (4618, 0.3044154920548746),
  (3206, 0.3011337361161691),
  (4000, 0.2856944275418137),
  (4510, 0.2789229147297112),
  (2126, 0.2647501252423849),
  (1953, 0.2640298303297865),
  (3515, 0.25384559041309324),
  (4965, 0.250098552715766),
  (3932, 0.24974682

In [67]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))